In [ ]:
%matplotlib inline
from __future__ import division
from data import DatFile, Data2D
import matplotlib.pyplot as plt
import itertools
from scipy import signal, ndimage, optimize
from scipy.optimize import leastsq
from matplotlib import cm
import numpy as np


class SimpleNamespace(object):
    """A simple container for parameters."""
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


def prepare_data(filename, v_min, v_max, chop=True):
    data = np.load(filename).item()
    
    # The abs z array may contain a (now useless) mask, which we discard
    if isinstance(data['z'], np.ma.core.MaskedArray):
        data['z'] = data['z'].data
    data['x'] *= 1e6 * 0.4835 # Convert voltage axis to µV and then to GHz
    
    # Chop data to include only desired region
    if chop:
        for key, val in data.iteritems():
            a = val[:,v_min:v_max] # the indeces at which we chop are chosen by eye
            data[key] = a
    
    # We only need one array for field and voltage values
    y = data['y'][:,0] 
    x = data['x'][0] ## NOTE: THIS MUST BE CORRECTED FOR THE DRIFT!
    data['x'] = x
    data['y'] = y
    return data


def plot_datascan(d):
    z = d['z'].T[::-1]
    x = d['x']
    y = d['y']
    
    dx, dy = x[1]-x[0], y[1]-y[0]
    ranges = [y[0]-dy/2, y[-1]+dy/2, x[0]-dx/2, x[-1]+dx/2]
    
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.imshow(z, extent = ranges, aspect='auto', interpolation='none', cmap='gist_heat_r')
    cbar = fig.colorbar(cax)
    ax.autoscale(False)


def calibration_phase_axis(data1, data2, i1=2, e1=12, i2=20, e2=35, plot=True):
    z1 = data1['z']
    z2 = data2['z']
    left1 = z1[i1:e1]
    left2 = z2[i2:e2]
    # find most symmetric slice on right part of the dataset
    idx1 = np.argmin([np.linalg.norm(left1-np.flipud(z1[n:n+(e1-i1)])) for n in xrange(len(z1)-(e1-i1))])
    idx2 = np.argmin([np.linalg.norm(left2-np.flipud(z2[n:n+(e2-i2)])) for n in xrange(len(z2)-(e2-i2))])
    # Find Bpi
    x1, x2 = data1['y'][e1-1], data1['y'][idx1]
    Bpi = x1 + (x2 - x1) / 2
    if plot:
        plot_datascan(data1)
        plt.axvline(Bpi)
        plt.show()
    # Find B0
    x1, x2 = data2['y'][e2-1], data2['y'][idx2]
    B0 = x1 + (x2 - x1) / 2
    if plot:
        plot_datascan(data2)
        plt.axvline(B0)
        plt.show()
    a, b =  - 2 * (Bpi - B0), B0
    for data in [data1, data2]:
        data['y'] -= b
        data['y'] /= a
        data['y'] *= 2*np.pi
    return a, b


def peak_positions(data):
    return np.array([data['x'][np.argmax(d)] for d in data['z']])

In [30]:
def merge_points(line_list):
    # A problem with the points is that, due to finite resolution,
    # some consecutive points have equal values of voltage. We merge these points
    # by taking their average x.
    lines = sum(line_list, [])
    new_lines = []
    for k, group in itertools.groupby(lines, lambda l: l[1]):
        xs = [g[0] for g in group]
        new_lines.append((np.mean(xs), k))
    return new_lines

In [34]:
def potential(phi, p, s='0'):
    r = np.sqrt(1 - p.T) # use reflection amplitude in the formulas, for brevity
    if s == 'z':
        phi = p.flux - phi
        return np.cos(0.5 * r * phi) * np.cos(0.5 * phi) + r * np.sin(0.5 * r * phi) * np.sin(0.5 * phi)
    elif s  == 'y':
        phi = p.flux - phi
        return r * np.cos(0.5 * r * phi) * np.sin(0.5 * phi) - np.sin(0.5 * r * phi) * np.cos(0.5 * phi)
    elif s == '0':
        return (1 - np.cos(phi))
    else:
        raise ValueError('wrong value of s')


def n_me(M, dk):
    x = np.pi * dk / M
    return (-1)**dk  / (np.tan(x) * np.sin(x)) / 2


def create_hamiltonian(p, M, sparse=False):
    # offset diagonal term from charging energy
    h0 = p.Ec * (M**2 - 1) / 12 * np.eye(M) + 0j
    
    # cosine part of the potential
    ks = np.arange(-(M-1)/2, (M+1)/2, 1)
    h0 += p.Ej * np.diag(np.array([potential(2 * np.pi * k / M, p, '0') for k in ks])) + 0j
     
    #off diagonal parts of the charging energy
    for dk in range(1, M):
        h0 += p.Ec * n_me(M, dk) * (np.eye(M, k=dk) + np.eye(M, k=-dk)) + 0j
    
    # z, y parts of the potential
    hz = p.Ea * np.diag(np.array([potential(2 * np.pi * k / M, p, 'z') for k in ks])) + 0j
    hy = p.Ea * np.diag(np.array([potential(2 * np.pi * k / M, p, 'y') for k in ks])) + 0j
    h = np.bmat([[h0 + hz, -1j * hy], [1j * hy, h0 - hz]])
    if sparse:
        return sp.coo_matrix(h)
    else:
        return np.array(h)

def find_optimal_M(p, max_tol = 9e-8, verbose=False):
    M = 3
    tol = 100
    H = create_hamiltonian(p, M, sparse=False)
    try:
        ens = np.linalg.eigvalsh(H)
    except np.linalg.LinAlgError:
            print "Error!!"
            print M, p.Ej, p.Ec, p.Ea, p.T, p.flux
            return H
    ens = np.linalg.eigvalsh(H)
    de = ens[1]-ens[0]
    while tol > max_tol:
        M += 4
        H = create_hamiltonian(p, M, sparse=False)
        try:
            ens = np.linalg.eigvalsh(H)
        except np.linalg.LinAlgError:
            print "Error!!"
            print M, p.Ej, p.Ec, p.Ea, p.T, p.flux
            return H
        de_new = ens[1]-ens[0]
        tol = np.abs(de_new - de)
        if verbose:
            print M, de, de_new, tol
        de = de_new
    return M

def current_operator(p, sparse=False):
    #if hasattr(p, 'M')==False:
    M = p.M if hasattr(p, 'M') else find_optimal_M(p)
    
    ks = np.arange(-(M-1)/2, (M+1)/2, 1)
    I0 = p.Ej * np.diag(np.array([potential_derivative(2 * np.pi * k / M, p, '0') for k in ks])) + 0j
    Iz = p.Ea * np.diag(np.array([potential_derivative(2 * np.pi * k / M, p, 'z') for k in ks])) + 0j
    Iy = p.Ea * np.diag(np.array([potential_derivative(2 * np.pi * k / M, p, 'y') for k in ks])) + 0j
    
    I = np.bmat([[I0 + Iz, -1j * Iy], [1j * Iy, I0 - Iz]])
    if sparse:
        return sp.coo_matrix(I)
    else:
        return np.array(I)
    
def potential_derivative(phi, p, s='0'):
    r = np.sqrt(1 - p.T) # use reflection amplitude in the formulas, for brevity
    if s == 'z':
        phi = p.flux - phi
        return - 0.5 * p.T * np.sin(0.5 * phi) * np.cos(r * 0.5 * phi)
    elif s  == 'y':
        phi = p.flux - phi
        return + 0.5 * p.T * np.sin(0.5 * phi) * np.sin(r * 0.5 * phi)
    elif s == '0':
        return np.sin(phi)
    else:
        raise ValueError('wrong value of s')
        
        
def rs_abs_model(plist, xs, ys, idx, Ej=None, Ec=None, Ea=None, max_tol=9e-8, sigmas=None):
    """ Returns the list of residuals between the data points
    and the transition energies, given a list of Hamiltonian parameters.

    Parameters:
    -----------
    plist : list of floats
        List of Hamiltonian parameters (excluding flux)
    xs : 1d array
        X values of the data.
    ys : 1d array
        Y values of the data.
    sigmas: 1d array
        Y data errors, if available.
    M : int
        Size of Hamiltonian. Must be odd.
    idx : list
        Index which marks the beginning of data corresponding
        to the ABS transition line. It is used to associate each
        data point to the correct transition energy.

    Return:
    ------
    1d array of weighted residuals.
    """
    p = SimpleNamespace()
    p.flux = np.pi # Initalize value of flux to compute optimal matriz size
    
    if not Ej and not Ec and not Ea:
        p.Ec, p.Ej, p.Ea, p.T = plist
    elif not Ec and not Ea:
        p.Ec, p.Ea, p.T = plist
        p.Ej = Ej
    elif not Ea:
        p.Ea, p.T = plist
        p.Ej, p.Ec = Ej, Ec
    else:
        p.T = plist[0]
        p.Ej, p.Ec, p.Ea = Ej, Ec, Ea
    
    # if leastsq attempts T > 1, a huge residual is returned
    if int(np.floor(p.T)) is not 0:
        return 1e12 * ys
    # Constraints values of other energy scales
    if p.Ea > 50 or p.Ea < 0:
        return 1e12 * ys
    if p.Ej > 50 or p.Ej < 0:
        return 1e12 * ys
    if p.Ec > 10 or p.Ec < 0:
        return 1e12 * ys
    
    print p.Ej, p.Ec, p.Ea
    print "Determining matrix size M..."
    M = find_optimal_M(p, max_tol = max_tol, verbose=True)
    print "Parameter values:"
    print p.Ec, p.Ej, p.Ea, p.T, M
    hams = []
    for x in xs:
        p.flux = x
        hams.append(create_hamiltonian(p, M, sparse=False))
    evals = [np.sort(np.linalg.eigvalsh(h)) for h in hams]
    freqs = [(ev - ev[0])[1:] for ev in evals]
    
    # Chooses which energy level is closest to the ABS line.
    # Note that it may depend on initial parameters.
    n = np.argmin(np.abs(freqs[idx] - ys[idx]))
    print "Picking which is the ABS line..."
    print freqs[0][:3]
    print freqs[idx][:3], ys[idx], n
    
    frequencies = np.hstack(([f[0] for f in freqs[:idx]],
                             [f[n] for f in freqs[idx:]]))
    weights = (1. / sigmas) if sigmas else 1.
    print "Sum of residues of current iteration:"
    print np.sum(np.abs(frequencies - ys))**2
    print "Going to next iteration...", "\n\n"
    return (frequencies - ys) * weights


def fit_abs_model(lines, idx, p0, Ej=None, Ec=None, Ea=None, max_tol=9e-7):
    """
    Returns the best estimate for the parameters p0 and their
    covariance matrix.

    The obtain the covariance matrix, the jacobian around the
    solution returned by leastsq is multiplied by the residual variance

    s = sum(rs) / (n-m)

    where n is the number of data points and m the number of fitting
    parameters. This is the same as what is done in scipy.optimize.curve_fit.
    
    Parameters:
    -----------
    lines : list
        List of points to be fitted, one list for every
        different transition.
    idx: int
        Index which marks the beginning of data corresponding
        to the ABS transition line.
    p0 : list
        Initial guess for the fitting parameters.
    Ej : float
        Josephson energy of the conventional junction.
        If Ej=None, Ej is passed via p0 and included as fitting parameter.
        Otherwise, it is kept as a fixed parameter.
    Ec : float
        Charging energy. Same instructions as Ej.
    """
    fit_result = {'initial_guess' : p0}
    xs, ys= np.asarray(zip(*lines))
    #lengths = [len(l) for l in lines]
    #idxs = [sum(lengths[:n]) for n in np.arange(1, 7)]
    (popt, pcov, infodict, errmsg, ier) = leastsq(rs_abs_model, x0=p0,
                                                  args=(xs, ys, idx, Ej, Ec, Ea, max_tol), full_output=1)
    #chi_squared = (np.asarray(rs_abs_model(popt, xs, ys, idx, Ej, Ec, Ea, max_tol))**2).sum()
    #chi_squared /= (len(xs) - len(popt))
    #pcov = pcov * chi_squared
    std_devs = np.sqrt(np.diag(pcov))
    fit_result['popt'] = popt
    #fit_result['chi_squared'] = chi_squared
    fit_result['infodict'] = infodict
    fit_result['pcov'] = pcov
    fit_result['std_devs'] = std_devs
    fit_result['errmsg'] = errmsg
    fit_result['ier'] = ier
    return fit_result


def plot_fit_results(res, lines, Ej=None, Ec=None, Ea=None, n_fluxes=100,
                     flux_in=-2*np.pi, flux_final=3*np.pi, plot_current=False, plot_wfs=False):
    p =  SimpleNamespace()
    if not Ej and not Ec and not Ea:
        p.Ec, p.Ej, p.Ea, p.T = res['popt']
    elif not Ec and not Ea:
        p.Ec, p.Ea, p.T = res['popt']
        p.Ej = Ej
    elif not Ea:
        p.Ea, p.T = res['popt']
        p.Ej, p.Ec = Ej, Ec
    else:
        p.T = res['popt'][0]
        p.Ej, p.Ec, p.Ea = Ej, Ec, Ea
    
    p.flux = np.pi
    p.M = find_optimal_M(p, max_tol=5e-6, verbose=False)
    
    fluxes = np.linspace(flux_in, flux_final, n_fluxes)
    evals = []
    matrix_elements = []
    for (n, flux) in enumerate(fluxes):
        p.flux = flux
        evs, evec = np.linalg.eigh(create_hamiltonian(p, p.M))
        evals.append(np.sort(evs))
        wavefunctions = evec.T
        iop = current_operator(p)
        gs_i_n = np.array([np.abs(np.conj(wavefunctions[0]).T.dot(iop).dot(wf))**2 for wf in wavefunctions])
        #gs_i_n /= gs_i_n[1]
        matrix_elements.append(gs_i_n[1:8])
    
    freqs = np.array([(ev - ev[0])[1:] for ev in evals])
    matrix_elements = np.array(matrix_elements)
    matrix_elements /= matrix_elements[0,0]  # normalize matrix elements by the 0-->1 one
    abs_energies = np.array([2 * p.Ea * np.sqrt(1 - p.T * np.sin(0.5 * flux)**2) for flux in fluxes])
    
    # color array
    ncol = 5
    cmap = cm.brg
    col_ind = np.linspace(20, 180, ncol)
    colors = cmap(col_ind.astype(int))
    
    ## Plot of the fit
    fig, ax = plt.subplots(figsize=(8,8))
    for (f, col) in zip(freqs.T[:5], colors):
        ax.plot(fluxes, f, c=col)
    phase, frequency = zip(*lines)
    ax.plot(fluxes, abs_energies, 'k--') # plot of 'bare' ABS energy for comparison
    ax.scatter(phase, frequency)
    ax.set_xlim((-2*np.pi, 3*np.pi))
    ax.set_title(r'$E_C=%.4s, E_J=%.4s, E_A=%.4s, T=%.4s$' % (p.Ec, p.Ej, p.Ea, p.T))
    fig.show()
    
    ## Plot of current operator matrix elements
    if plot_current:
        fig, ax = plt.subplots()
        for n, (m, col) in enumerate(zip(matrix_elements.T, colors)):
            ax.plot(fluxes, m, c=col, label=n+1)
        ax.legend()
        ax.set_xlim((flux_in, flux_final))
        ax.set_yscale('log')
        ax.set_ylim((1e-2, 3))
        fig.show()
    
    ## Plot of the wavefunctions at zero flux
    if plot_wfs:
        p.flux = 0.
        _, evec = np.linalg.eigh(create_hamiltonian(p, p.M))
        wavefunctions = evec.T
        for (n, wf) in enumerate(wavefunctions[:6]):
            L = len(wf)
            fig, ax = plt.subplots(figsize=(5,2))
            ax.plot(np.abs(wf)**2, 'k.')
            ax.set_xlim(0, L)
            plt.axvline(L/2, c='k')
            ax.set_ylabel(r'$|\Psi(\phi)|^2$')
            ax.set_xticks([L/8, L/4, 3*L/8, 5*L/8, 3*L/4, 7*L/8])
            ax.set_xticklabels([r'$-\pi/2$', r'$0$', r'$-\pi/2$', r'$-\pi/2$', r'$0$', r'$-\pi/2$'])
            ax.set_yticks([])
            ax.set_title(r'Eigenstate n. %s' % n)
            fig.show()